In [3]:
from google.colab import files
uploaded = files.upload()

# Step 2: Check the filename
filename = list(uploaded.keys())[0]
print("Uploaded file:", filename)

# Step 3: Load based on file type
import pandas as pd

if filename.endswith('.csv'):
    df = pd.read_csv(filename)
elif filename.endswith('.xlsx'):
    df = pd.read_excel(filename)
else:
    raise ValueError("Unsupported file type")

# Step 4: Preview the data
df.head()


Saving zomato.csv to zomato.csv
Uploaded file: zomato.csv


,Unnamed: 0.1,Unnamed: 0,restaurant name,restaurant type,rate (out of 5),num of ratings,avg cost (two people),online_order,table booking,cuisines type,area,local address
0,0,0,#FeelTheROLL,Quick Bites,3.4,7,200.0,No,No,Fast Food,Bellandur,Bellandur
1,1,1,#L-81 Cafe,Quick Bites,3.9,48,400.0,Yes,No,"Fast Food, Beverages","Byresandra,Tavarekere,Madiwala",HSR
2,2,2,#refuel,Cafe,3.7,37,400.0,Yes,No,"Cafe, Beverages",Bannerghatta Road,Bannerghatta Road
3,3,3,'@ Biryani Central,Casual Dining,2.7,135,550.0,Yes,No,"Biryani, Mughlai, Chinese",Marathahalli,Marathahalli
4,4,4,'@ The Bbq,Casual Dining,2.8,40,700.0,Yes,No,"BBQ, Continental, North Indian, Chinese, Bever...",Bellandur,Bellandur


Building the dashboard for the Data Taken from Zomato  using DASH

In [6]:
# Show actual column names
print("🔍 Column names in your dataset:")
print(df.columns.tolist())


🔍 Column names in your dataset:
['Unnamed: 0.1', 'Unnamed: 0', 'restaurant name', 'restaurant type', 'rate (out of 5)', 'num of ratings', 'avg cost (two people)', 'online_order', 'table booking', 'cuisines type', 'area', 'local address']


In [10]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px

# Load dataset
df = pd.read_csv("zomato.csv")  # Replace with your filename

# Rename columns for easier access (optional)
df = df.rename(columns={
    'restaurant name': 'name',
    'rate (out of 5)': 'rate',
    'num of ratings': 'votes',
    'avg cost (two people)': 'cost',
    'online_order': 'online_order',
    'area': 'location',
    'cuisines type': 'cuisines'
})

# Clean data
df = df.dropna(subset=['location', 'rate', 'votes', 'cost', 'cuisines'])
df['rate'] = df['rate'].astype(str).str.replace("/5", "").str.strip()
df = df[df['rate'] != 'NEW']  # Remove "NEW" or non-numeric entries
df['rate'] = df['rate'].astype(float)
df['votes'] = df['votes'].astype(int)
df['cost'] = df['cost'].astype(str).str.replace(",", "").astype(float)

# Create Dash app
app = dash.Dash(__name__)

# Pie chart: Online order availability
online_fig = px.pie(df, names='online_order', title='Online Order Availability')

# Bar chart: Top 10 locations by restaurant count
top_areas = df['location'].value_counts().nlargest(10)
area_fig = px.bar(x=top_areas.index, y=top_areas.values,
                  labels={'x': 'Location', 'y': 'Number of Restaurants'},
                  title='Top 10 Areas with Most Restaurants')

# Scatter plot: Cost vs Rating
scatter_fig = px.scatter(
    df,
    x='cost', y='rate',
    size='votes', color='online_order',
    hover_data=['name', 'location', 'cuisines'],
    title='Cost for Two vs Rating (Bubble = Votes)'
)

# Dashboard layout
app.layout = html.Div([
    html.H1("Zomato Bangalore Dashboard", style={'textAlign': 'center'}),

    dcc.Graph(figure=online_fig),
    dcc.Graph(figure=area_fig),
    dcc.Graph(figure=scatter_fig),
])

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>

Insight 1: “Online ordering is available in over 60% of restaurants”

Insight 2: “Most restaurants are in Koramangala and BTM”

Insight 3: “Cost for two does not directly correlate with rating”
